In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os.path
from PIL import Image
from imblearn.under_sampling import RandomUnderSampler
import random

In [2]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import auc, roc_curve

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.optim import AdamW, Adam
from torchvision.transforms import transforms, v2
from torchvision.models import vgg16
import torchvision.models as models

# settings 

In [4]:
device = ("cuda" if torch.cuda.is_available() 
          else "mps" if torch.backends.mps.is_available() 
          else "cpu")
print(f"device: {device}")

device: mps


In [5]:
torch.manual_seed(1729)
device_gen = torch.Generator(device)
torch.set_default_device(device)

In [6]:
IN_KAGGLE = "KAGGLE_KERNEL_RUN_TYPE" in os.environ
IN_COLAB = "COLAB_GPU" in os.environ

if IN_COLAB:
    from google.colab import userdata    
    # only the first time
    # os.environ["KAGGLE_KEY"] = "d0ebb5786a5a5439881827924cf0ccbd"
    # os.environ["KAGGLE_USERNAME"] = "yuda03979"
    # 
    # ! kaggle competitions download isic-2024-challenge
    # ! unzip isic-2024-challenge.zip
    IMG_DIR = "/content/train-image/image"
    CSV_PATH = "/content/train-metadata.csv"
elif IN_KAGGLE:
    IMG_DIR = "/kaggle/input/isic-2024-challenge/train-image/image"
    CSV_PATH = "/kaggle/input/isic-2024-challenge/train-metadata.csv"
else:
    IMG_DIR = "/Users/yuda/Desktop/data_bases/isic-2024-challenge/train-image/image"
    CSV_PATH = "/Users/yuda/PycharmProjects/my_isic2024/isic-2024-challenge/train-metadata.csv"

In [7]:
torch.manual_seed(1729)

In [8]:
def auc_80(y_pred, y):
    fpr, tpr, thresholds = roc_curve(y, y_pred)
    tpr_80 = [0.80 if i >= 0.80 else i for i in tpr]
    return auc(fpr, tpr) - auc(fpr, tpr_80)

# data and constance

In [9]:
BATCH_SIZE = 32
NUM_EPOCHS = 10
IMG_SIZE = (224, 224)
NUM_CLASSES = 1
LR = 1e-4
RANDOM_STATE = 40
CLASS_1_AMOUNT_PER_BATCH = BATCH_SIZE // 2

In [10]:
ISIC_df = pd.read_csv(CSV_PATH)
ISIC_df = ISIC_df[["isic_id", "patient_id", "target"]]
ISIC_df["img_path"] = ISIC_df["isic_id"].apply(lambda id: f"{os.path.join(IMG_DIR, id)}.jpg")
ISIC_df.drop(["isic_id"], axis=1, inplace=True)
ISIC_df

/var/folders/tw/t4_6l2cn5hz3hqm6tjzgwk5c0000gn/T/ipykernel_7213/2637853055.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  ISIC_df = pd.read_csv(CSV_PATH)


,patient_id,target,img_path
0,IP_1235828,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
1,IP_8170065,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
2,IP_6724798,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
3,IP_4111386,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
4,IP_8313778,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
...,...,...,...
401054,IP_1140263,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
401055,IP_5678181,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
401056,IP_0076153,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
401057,IP_5231513,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...


In [11]:
transform = v2.Compose([
    v2.Resize(IMG_SIZE),
    v2.ToDtype(torch.float32, scale=True),
    v2.ToTensor(),
    v2.Normalize(mean = [0.6298, 0.5126, 0.4097], std = [0.1386, 0.1308, 0.1202]),
])

def augmentation_transform():
    transform = [v2.RandomRotation(90, expand=False),
                v2.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=3),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
                v2.CenterCrop(150),]
    return random.choice(transform)

/opt/anaconda3/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


# split according patients

In [12]:
def initialize_dataset_10k_0(CSV_PATH):
    '''
    return two data frames with 10k target == 0, and all the target == 1.
    train have 3/4 target == 1, and 1/4 in the val_df. each patients
    :param CSV_PATH: path to the csv (train)
    :return: (df, df); train and val
    '''

    # reading dataset
    ISIC_df = pd.read_csv(CSV_PATH)
    ISIC_df = ISIC_df[["isic_id", "patient_id", "target"]]
    ISIC_df["img_path"] = ISIC_df["isic_id"].apply(lambda id: f"{os.path.join(IMG_DIR, id)}.jpg")
    ISIC_df.drop(["isic_id"], axis=1, inplace=True)

    # splitting into target == 1 and target == 0
    class_1_df = ISIC_df[ISIC_df["target"] == 1].reset_index(drop=True)
    class_0_df = ISIC_df[ISIC_df["target"] == 0].reset_index(drop=True)

    def split_according_patients(df, train_size, drop_patients=True):
        X, y, groups = df, df["target"], df["patient_id"]

        gss = GroupShuffleSplit(n_splits=1, train_size=train_size, random_state=RANDOM_STATE)
        gss_gen = gss.split(X, y, groups)
        train_index, val_index = next(gss_gen)
        if drop_patients:
            train_df = df.loc[train_index, ["img_path", "target"]].reset_index(drop=True)
            val_df = df.loc[val_index, ["img_path", "target"]].reset_index(drop=True)
        else:
            train_df = df.loc[train_index, ["img_path", "patient_id", "target"]].reset_index(drop=True)
            val_df = df.loc[val_index, ["img_path", "patient_id", "target"]].reset_index(drop=True)

        print(f"len train_df: {len(train_df)}\nlen val_df: {len(val_df)}")
        return train_df, val_df

    train_class_1_df, val_class_1_df = split_according_patients(class_1_df, 0.75, False)

    def take_common_pationts(class_0_df, class_1_df):
        '''
        takes the patients that in class_0_df and class_1_df both.
        :param class_0_df: target == 0
        :param class_1_df: target == 1
        :return: only the patients that in both df
        '''
        class_1_patients = class_1_df["patient_id"].unique()
        class_0_df = class_0_df[class_0_df["patient_id"].isin(class_1_patients)]
        return pd.concat((class_0_df, class_1_df), axis=0).reset_index(drop=True)

    train_df = take_common_pationts(class_0_df, train_class_1_df)
    val_df = take_common_pationts(class_0_df, val_class_1_df)
    
    return train_df, val_df



def get_dataloader(df):
    dataset = ISICDataset(train_df, transform, augmentation_transform)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    return dataloader 



def get_dataloader_with_undersampling(train_df, val_df, train_class_0_size=10000, val_class_0_size=10000):
    train_class_1_size, val_class_1_size = len(train_df[train_df['target'] == 1]), len(val_df[val_df['target'] == 1])
    train_df.sample(frac=1)
    # under sampling
    undersample = RandomUnderSampler(sampling_strategy={0: train_class_0_size, 1: train_class_1_size})
    train_df, _ = undersample.fit_resample(train_df, train_df["target"])
    train_df = train_df.reset_index(drop=True)
    
    undersample = RandomUnderSampler(sampling_strategy={0: val_class_0_size, 1: val_class_1_size})
    val_df, _ = undersample.fit_resample(val_df, val_df["target"])
    val_df = val_df.reset_index(drop=True).iloc[::-1].reset_index(drop=True) # to make al the class_1 in the beginning
    
    class_1_train_df = train_df[train_df["target"] == 1]
    class_0_train_df = train_df[train_df["target"] == 0]
    
    
    train_ds = ISICDataset(train_df, transform=transform, augmentation_transform=augmentation_transform)
    val_ds = ISICDataset(val_df, transform=transform)
    
    sampler = BalancedBatchSampler(class_0_train_df, class_1_train_df, batch_size=BATCH_SIZE, class_1_amount_per_batch=CLASS_1_AMOUNT_PER_BATCH)
    
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, generator=device_gen)
    val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, generator=device_gen)
    
    return train_dl, val_dl

# ISIC_Dataset class

In [13]:
class ISICDataset(Dataset):
    
    def __init__(self, df, transform=None, augmentation_transform=None):
        self.df = df
        self.transform = transform
        self.augmentation_transform = augmentation_transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img = Image.open(self.df.loc[idx, "img_path"]).convert("RGB")
        img = v2.Resize(IMG_SIZE)(img)
        img = v2.ToDtype(torch.float32, scale=True)(img)
        label = self.df.loc[idx, "target"]
        if self.augmentation_transform:    
            img = self.augmentation_transform()(img) if random.random() > 0.2 else img
        img = self.transform(img)
        # if random.random() > 0.98:
        #     plt.imshow(img.permute(1, 2, 0))
        #     plt.show()
        return img, label



In [14]:
class BalancedBatchSampler(Sampler):

    def __init__(self, class_0_df, class_1_df, batch_size, class_1_amount_per_batch=10):
        self.class_0_df = class_0_df
        self.class_1_df = class_1_df

        self.batch_size = batch_size
        self.class_1_amount_per_batch = class_1_amount_per_batch
        self.num_batches = (len(self.class_0_df) + len(self.class_1_df)) // batch_size

    def __iter__(self):
        class_0_iter = iter(self.class_0_df.index)
        class_1_iter = iter(self.class_1_df.index)
        
        for _ in range(self.num_batches):
            batch = []
            try:
                for _ in range(self.class_1_amount_per_batch):
                    batch.append(next(class_1_iter))
            except:
                class_1_iter = iter(self.class_1_df.index)
                batch = []
                for _ in range(self.class_1_amount_per_batch):
                    batch.append(next(class_1_iter))
            try:
                for _ in range(self.batch_size - self.class_1_amount_per_batch):
                    batch.append((next(class_0_iter)))
            except:
                class_0_iter = iter(self.class_0_df.index)
                for _ in range(self.batch_size - self.class_1_amount_per_batch):
                    batch.append(next(class_0_iter))
            random.shuffle(batch)
            for idx in batch:
                yield idx

    def __len__(self):
        return self.num_batches


# model

In [15]:
def load_model(model_name="mobilenet_v3_small", num_classes=2):

    if model_name == "mobilenet_v3_small":
        model = models.mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.DEFAULT)
        model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
    
    elif model_name == "efficientnet_v2_m":
        model = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    
    elif model_name == "efficientnet_b0":
        model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    
    elif model_name == "vgg16":
        model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    
    elif model_name == "resnet50":
        model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    elif model_name == "resnet18":
        model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    elif model_name == "vit_b_16":
        model = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)
        in_features = model.heads.head.in_features
        model.heads.head = nn.Linear(in_features, num_classes)
    else:
        raise ValueError(f"Model {model_name} is not supported yet")
        
#     print(model)
    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss() if num_classes > 1 else nn.BCELoss()

    return model, optimizer, criterion


In [16]:
model, optimizer, criterion = load_model(num_classes=NUM_CLASSES)

Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /Users/yuda/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:08<00:00, 1.17MB/s]


# training func

In [17]:
def train(dataloader, model, optimizer, criterion):
    model.train()
    total_loss = 0
    sum_batches = 0
    for batch, (imgs, labels) in enumerate(dataloader):
        sum_batches += 1
        imgs, labels = imgs.to(device), labels.to(device)
        if NUM_CLASSES == 1:
            labels = labels.unsqueeze(1).float()
            pred_labels = model(imgs)
            pred_labels = torch.sigmoid(pred_labels)
        elif NUM_CLASSES == 2:
            pred_labels = model(imgs)
        else:
            raise Exception(f"define NUM_CLASSES")
        loss = criterion(pred_labels, labels)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        total_loss += loss.item()
        if batch % 10 == 0:
            print(f"{batch + 1} from {len(dataloader) * BATCH_SIZE}, loss: {loss}, auc: -")
    return total_loss / sum_batches


In [18]:
def test(dataloader, model, criterion):
    model.eval()
    pred_arr = [0, 1] # initialize for cases where only class_0 or only class_1 
    labels_arr = [1, 0]
    total_loss = 0

    with torch.no_grad():
        for batch, (imgs, labels) in enumerate(dataloader):
            imgs, labels = imgs.to(device), labels.to(device)
            if NUM_CLASSES == 1:
                labels = labels.unsqueeze(1).float()
    
                pred_labels = model(imgs)
                pred_labels = torch.sigmoid(pred_labels)
                loss = criterion(pred_labels, labels)
                
                total_loss += loss.item()
                pred_arr = np.concatenate((pred_arr, pred_labels.cpu().flatten().detach().numpy()), axis=None)
                labels_arr = np.concatenate((labels_arr, labels.cpu().flatten().detach().numpy()), axis=None)
            elif NUM_CLASSES == 2:
                # labels = labels.unsqueeze(1).float()
                pred_labels = model(imgs)
                # pred_labels = torch.sigmoid(pred_labels)
                loss = criterion(pred_labels, labels)
                
                total_loss += loss.item()
                
                probabilities = nn.functional.softmax(pred_labels, dim=1)
                # _, predicted_classes = torch.max(probabilities, dim=1)
                
                pred_arr = pred_arr + probabilities[:, 1].cpu().flatten().detach().numpy().tolist()
                labels_arr = labels_arr + labels.detach().flatten().cpu().numpy().tolist()
            else:
                raise Exception(f"define NUM_CLASSES")
            
            if batch % 5 == 0:
                print(f"{batch + 1} from {len(dataloader)}, loss: {loss}, auc: {auc_80(pred_arr, labels_arr)}")
    return total_loss / len(dataloader), auc_80(pred_arr, labels_arr)

## initialize the data

In [19]:
train_df, val_df = initialize_dataset_10k_0(CSV_PATH)

/var/folders/tw/t4_6l2cn5hz3hqm6tjzgwk5c0000gn/T/ipykernel_7213/834590914.py:10: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  ISIC_df = pd.read_csv(CSV_PATH)


len train_df: 306
len val_df: 87


# start runing

In [20]:
train_loss_arr = []
test_loss_arr = []
test_auc_arr = []

In [21]:
gap = "------------------------------"

for epoch in range(NUM_EPOCHS):
    train_dataloader, val_dataloader = get_dataloader_with_undersampling(train_df, val_df, 2000, 2000)


    print(f"{gap}\nepoch: {epoch + 1}\n{gap}")
    
    print("train:")
    train_loss_arr.append(train(train_dataloader, model, optimizer, criterion))
    print("test:")
    loss, auc_ = test(val_dataloader, model, criterion)
    
    test_loss_arr.append(loss)
    test_auc_arr.append(auc_)
    print(f"train_loss_arr: {train_loss_arr[-1]} ; test_loss_arr: {test_loss_arr[-1]} ; test_auc_arr: {test_auc_arr[-1]} ;")
    # torch.save(model.state_dict(), f"model_{epoch + 1}.pth")
    # print(f"Saved PyTorch Model State to model_{epoch + 1}.pth")
print("done!")

------------------------------
epoch: 1
------------------------------
train:
1 from 96, loss: 0.7332229614257812, auc: -


TypeError: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.

In [ ]:
# torch.save(model.state_dict(), "model_.pth")
# print("Saved PyTorch Model State to model.pth")

In [ ]:
print(f"train_loss_arr: {train_loss_arr} ;\n test_loss_arr: {test_loss_arr} ;\n test_auc_arr: {test_auc_arr} ;")

In [ ]:
pd.DataFrame({"train_loss_arr":train_loss_arr, "test_loss_arr":test_loss_arr, "test_auc_arr":test_auc_arr})


In [ ]:
# to play with: class_1_amount_per_batch
#